# Migrate or clone a local lakeFS repository to AWS

#### Use this notebook if you want to migrate/clone a local source repository to a target repository on AWS

## Prerequisites

#### 1. Source and a target lakeFS environments (you can [deploy one independently](https://docs.lakefs.io/deploy/) or use the hosted solution [lakeFS Cloud](https://lakefs.cloud))
#### 2. Source repository
#### 3. S3 Bucket for target repository but don't create target repository ahead of time (you will create a target repo in this notebook)

## Setup Task: Import required Python packages

In [ ]:
%xmode Minimal
import lakefs
from lakefs.client import Client
import lakefs_sdk
from lakefs_sdk.client import LakeFSClient
import random
import os
import datetime
from awscliv2.api import AWSAPI

## Setup Task: Change your lakeFS credentials for the local source lakeFS environment

In [ ]:
sourceLakefsEndPoint = '<Source lakeFS Endpoint URL>' # e.g. 'http://host.docker.internal:8000' if you are running lakeFS in a separate Docker container on your local machine
sourceLakefsAccessKey = '<Source lakeFS Access Key>'
sourceLakefsSecretKey = '<Source lakeFS Secret Key>'

## Setup Task: Change your lakeFS credentials for the target lakeFS environment

In [ ]:
targetLakefsEndPoint = '<Target lakeFS Endpoint URL>' # e.g. 'https://username.aws_region_name.lakefscloud.io'
targetLakefsAccessKey = '<Target lakeFS Access Key>'
targetLakefsSecretKey = '<Target lakeFS Secret Key>'

## Setup Task: Change lakeFS repo names

In [ ]:
source_repo_name = "source-repo"
target_repo_name = "target-repo"

## Setup Task: Change main/production branch name for the source repo

In [ ]:
source_main_branch = "main"

## Setup Task: Change source and target storage namespace

In [ ]:
source_storage_namespace = 'local://localSourceFolder' # storage namespace for the local source repo
target_storage_namespace = 's3://myTargetBucket' # S3 bucket or any prefix/folder within the bucket e.g. 's3://myTargetBucket/<prefix>'

## Setup Task: Change AWS credentials

In [ ]:
aws_region = '<AWS region name>' # e.g. us-east-1
aws_access_key_id = 'aaaaaaaaaaaaa'
aws_secret_access_key = 'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb'

## Setup Task: Set AWS credentials for the default profile

In [ ]:
aws_api = AWSAPI()

aws_api.set_credentials("default", aws_access_key_id, aws_secret_access_key, "", aws_region)

## Setup Task: Create lakeFS Python client for source lakeFS environment

In [ ]:
if not 'source_lakefs_client' in locals():
    source_lakefs_client = Client(
        host=sourceLakefsEndPoint,
        username=sourceLakefsAccessKey,
        password=sourceLakefsSecretKey,
    )
    
    configuration = lakefs_sdk.Configuration(
        host=sourceLakefsEndPoint,
        username=sourceLakefsAccessKey,
        password=sourceLakefsSecretKey,
    )
    source_lakefs_sdk_client = LakeFSClient(configuration)
    
    print("Verifying lakeFS credentials…")
    try:
        v=source_lakefs_client.version
        sourceRepo = lakefs.Repository(source_repo_name, client=source_lakefs_client)
    except:
        print("🛑 failed to get lakeFS version")
    else:
        print(f"…✅lakeFS credentials verified\n\nℹ️lakeFS version {v}")

## Setup Task: Create lakeFS Python client for target lakeFS environment

In [ ]:
if not 'target_lakefs_client' in locals():
    target_lakefs_client = Client(
        host=targetLakefsEndPoint,
        username=targetLakefsAccessKey,
        password=targetLakefsSecretKey,
    )
    
    configuration = lakefs_sdk.Configuration(
        host=targetLakefsEndPoint,
        username=targetLakefsAccessKey,
        password=targetLakefsSecretKey,
    )
    target_lakefs_sdk_client = LakeFSClient(configuration)
    
    print("Verifying lakeFS credentials…")
    try:
        v=target_lakefs_client.version
    except:
        print("🛑 failed to get lakeFS version")
    else:
        print(f"…✅lakeFS credentials verified\n\nℹ️lakeFS version {v}")

# Step 1 - Commit Changes

## IMPORTANT: Uncommitted data is not migrated over so check uncommitted data (this might take time if you have many branches in the source repo)

In [ ]:
for branchList in sourceRepo.branches():
    for diff in sourceRepo.branch(branchList.id).uncommitted():
        print('Branch with uncommitted data: ' + branchList.id)

## OPTIONAL: Commit any uncommitted data in your source repo (this might take time if you have many branches in the source repo)
#### If you want, you can manually commit the changes for the branches listed above from the lakeFS UI

#### You can run previous command again after finishing this to verify that there are not any uncommitted data any more

In [ ]:
for branchList in sourceRepo.branches():
    for diff in sourceRepo.branch(branchList.id).uncommitted():
        ref = sourceRepo.branch(branchList.id).commit(message='Committed changes during the migration of the repository')
        print(ref.get_commit())

# Step 2 - Dump Metadata of Source Repository

In [ ]:
source_lakefs_sdk_client.internal_api.dump_refs(source_repo_name)

# Step 3 - Copy Data from Source to Target

#### You can directly copy data from local storage to target storage on your own
#### or you can run following printed command on your local machine to copy data from local Docker container to local machine first
#### (change the Docker container name for lakeFS and go to the folder where you cloned lakefs-samples Git repo before running the command)

In [ ]:
lakefs_docker_container_name = 'lakefs'
print('docker cp ' + lakefs_docker_container_name + ':/home/lakefs/lakefs/data/block/' + source_storage_namespace.split('://')[1] + \
      '/ lakeFS-samples/01_standalone_examples/migrate-or-clone-repo/localDownloadedSourceFolder/')

#### Now copy data from your local machine to target storage on S3

In [ ]:
s3SyncCommand = 'aws s3 sync ./localDownloadedSourceFolder/ ' + target_storage_namespace
print(s3SyncCommand)

! $s3SyncCommand

## Step 4 - Create Target Bare Repository

In [ ]:
lakefs.Repository(target_repo_name, client=target_lakefs_client).create(storage_namespace=f"{target_storage_namespace}", default_branch=source_main_branch, bare=True)

## Step 5 - Restore Metadata to Target Repository

In [ ]:
target_lakefs_sdk_client.internal_api.restore_refs(target_repo_name, source_lakefs_sdk_client.internal_api.dump_refs(source_repo_name))

## More Questions?

###### Join the lakeFS Slack group - https://lakefs.io/slack